In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import format_number

spark = SparkSession.builder.appName("LinReg").getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [4]:
df = spark.read.format('libsvm').load("data/bodyfat.txt")
df.show()

23/02/13 14:46:37 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
+------+--------------------+
| label|            features|
+------+--------------------+
|1.0708|(14,[0,1,2,3,4,5,...|
|1.0853|(14,[0,1,2,3,4,5,...|
|1.0414|(14,[0,1,2,3,4,5,...|
|1.0751|(14,[0,1,2,3,4,5,...|
| 1.034|(14,[0,1,2,3,4,5,...|
|1.0502|(14,[0,1,2,3,4,5,...|
|1.0549|(14,[0,1,2,3,4,5,...|
|1.0704|(14,[0,1,2,3,4,5,...|
|  1.09|(14,[0,1,2,3,4,5,...|
|1.0722|(14,[0,1,2,3,4,5,...|
| 1.083|(14,[0,1,2,3,4,5,...|
|1.0812|(14,[0,1,2,3,4,5,...|
|1.0513|(14,[0,1,2,3,4,5,...|
|1.0505|(14,[0,1,2,3,4,5,...|
|1.0484|(14,[0,1,2,3,4,5,...|
|1.0512|(14,[0,1,2,3,4,5,...|
|1.0333|(14,[0,1,2,3,4,5,...|
|1.0468|(14,[0,1,2,3,4,5,...|
|1.0622|(14,[0,1,2,3,4,5,...|
| 1.061|(14,[0,1,2,3,4,5,...|
+------+--------------------+
only showing top 20 rows



In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')
lr_model = lr.fit(df)

23/02/13 14:51:50 WARN Instrumentation: [6133500b] regParam is zero, which might cause numerical instability and overfitting.
23/02/13 14:51:50 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
lr_model.coefficients

DenseVector([-0.0022, 0.0, 0.0, -0.0, 0.0, 0.0001, -0.0002, 0.0001, -0.0001, -0.0, -0.0002, -0.0002, -0.0, 0.0004])

In [9]:
lr_model.intercept

1.0981463302283172

In [10]:
train, test = df.randomSplit([0.7, 0.3])

In [11]:
lr_model = lr.fit(train)

23/02/13 14:56:17 WARN Instrumentation: [c4a23d4d] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
test_results = lr_model.evaluate(test)

In [13]:
test_results.rootMeanSquaredError

0.004955289095027164

In [15]:
unlabeled_data = test.select('features')

In [16]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|(14,[0,1,2,3,4,5,...|  1.00826973518771|
|(14,[0,1,2,3,4,5,...| 1.020167641810168|
|(14,[0,1,2,3,4,5,...|1.0253768505578704|
|(14,[0,1,2,3,4,5,...|1.0265532108129278|
|(14,[0,1,2,3,4,5,...| 1.027210884332204|
|(14,[0,1,2,3,4,5,...|1.0270229682287173|
|(14,[0,1,2,3,4,5,...|1.0285355399346798|
|(14,[0,1,2,3,4,5,...|1.0302070498329485|
|(14,[0,1,2,3,4,5,...|1.0314249809154912|
|(14,[0,1,2,3,4,5,...|1.0313546073923379|
|(14,[0,1,2,3,4,5,...|1.0370764500660496|
|(14,[0,1,2,3,4,5,...|1.0385204709025855|
|(14,[0,1,2,3,4,5,...|1.0390197117424678|
|(14,[0,1,2,3,4,5,...|1.0402402882397581|
|(14,[0,1,2,3,4,5,...|1.0416181633948498|
|(14,[0,1,2,3,4,5,...|1.0427221778036426|
|(14,[0,1,2,3,4,5,...|1.0442142852870713|
|(14,[0,1,2,3,4,5,...|1.0440840392938042|
|(14,[0,1,2,3,4,5,...|  1.04482570471748|
|(14,[0,1,2,3,4,5,...|1.0464583634207476|
+--------------------+------------

In [44]:
# =======================

In [26]:
df = spark.read.format('csv').option("header", 'true').load("data/cruise_ship_info.csv")
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [27]:
df.groupBy("Cruise_line").count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [34]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_cat")
indexed = indexer.fit(df).transform(df)
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age='6', Tonnage='30.276999999999997', passengers='6.94', length='5.94', cabins='3.55', passenger_density='42.64', crew='3.55', Cruise_line_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age='6', Tonnage='30.276999999999997', passengers='6.94', length='5.94', cabins='3.55', passenger_density='42.64', crew='3.55', Cruise_line_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age='26', Tonnage='47.262', passengers='14.86', length='7.22', cabins='7.43', passenger_density='31.8', crew='6.7', Cruise_line_cat=1.0)]

In [31]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [35]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_cat']

In [74]:
from pyspark.sql.functions import col

assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 
                                       'cabins', 'passenger_density', 'Cruise_line_cat'], 
                            outputCol='features')

print(indexed.dtypes)
indexed = indexed.withColumn("Age", col("Age").cast("integer"))
indexed = indexed.withColumn("Tonnage", col("Tonnage").cast("double"))
indexed = indexed.withColumn("passengers", col("passengers").cast("double"))
indexed = indexed.withColumn("length", col("length").cast("double"))
indexed = indexed.withColumn("cabins", col("cabins").cast("double"))
indexed = indexed.withColumn("passenger_density", col("passenger_density").cast("double"))
indexed = indexed.withColumn("crew", col("crew").cast("double"))


print(indexed.dtypes)


[('Ship_name', 'string'), ('Cruise_line', 'string'), ('Age', 'int'), ('Tonnage', 'double'), ('passengers', 'double'), ('length', 'double'), ('cabins', 'double'), ('passenger_density', 'double'), ('crew', 'double'), ('Cruise_line_cat', 'double')]
[('Ship_name', 'string'), ('Cruise_line', 'string'), ('Age', 'int'), ('Tonnage', 'double'), ('passengers', 'double'), ('length', 'double'), ('cabins', 'double'), ('passenger_density', 'double'), ('crew', 'double'), ('Cruise_line_cat', 'double')]


In [75]:
output = assembler.transform(indexed)

In [76]:
output.select('features', 'crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [83]:
final_data = output.select(['features', 'crew'])
train, test = final_data.randomSplit([0.7, 0.3])
train.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              114|
|   mean| 7.99359649122808|
| stddev|3.701205287349327|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [84]:
test.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               44|
|   mean|           7.2775|
| stddev|2.904791650856171|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [85]:
lr_shp = LinearRegression(labelCol="crew")
trained_lr_shp = lr_shp.fit(train)

23/02/13 16:21:07 WARN Instrumentation: [aa997ef1] regParam is zero, which might cause numerical instability and overfitting.


In [86]:
ship_results = trained_lr_shp.evaluate(test)
ship_results.rootMeanSquaredError

0.6026843083370094